<a href="https://colab.research.google.com/github/sidsharma3/BiDirRNNHandWritingRecognition/blob/main/BiDirRNNHandWritingRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
from builtins import range, input
import os
from keras.models import Model
from keras.layers import Input, LSTM, Bidirectional, GlobalMaxPooling1D, Lambda, Concatenate, Dense
import keras.backend as K
import numpy as np
import pandas as pd

In [ ]:
def get_data(limit=None):
  df = pd.read_csv('./train.csv')
  data = df.values
  np.random.shuffle(data)
  X = data[:, 1:].reshape(-1, 28, 28) / 255.0 
  Y = data[:, 0]
  if limit is not None:
    X, Y = X[:limit], Y[:limit]
  return X, Y
X, Y = get_data()
D = 28 # since each image is 28 by 28 pixels
M = 20

In [ ]:
# since image is 28 pixels by 28 pixels we set D as 28
input_ = Input(shape=(D, D))
# vertical
rnn1 = Bidirectional(LSTM(M, return_sequences=True))
x1 = rnn1(input_) # output is N x D x 2M
x1 = GlobalMaxPooling1D()(x1) # output is N x 2M
# horizontal
rnn2 = Bidirectional(LSTM(M, return_sequences=True))
permutor = Lambda(lambda t: K.permute_dimensions(t, pattern=(0, 2, 1)))
x2 = permutor(input_)
x2 = rnn2(x2) 
x2 = GlobalMaxPooling1D()(x2)
# combining vertical and horizontal
concatenator = Concatenate(axis=1)
x = concatenator([x1, x2]) 
output = Dense(10, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)

In [ ]:
model.compile(
  loss='sparse_categorical_crossentropy', # since we do not want one hot encode data
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
r = model.fit(X, Y, batch_size=32, epochs=20, validation_split=0.3) 
# Best accuracy: 99.4%
# Best Validation accuracy: 98%

Epoch 1/20
919/919 [==============================] - 29s 32ms/step - loss: 0.3189 - accuracy: 0.9136 - val_loss: 0.2222 - val_accuracy: 0.9374
Epoch 2/20
919/919 [==============================] - 27s 30ms/step - loss: 0.1833 - accuracy: 0.9470 - val_loss: 0.1622 - val_accuracy: 0.9533
Epoch 3/20
919/919 [==============================] - 28s 30ms/step - loss: 0.1354 - accuracy: 0.9609 - val_loss: 0.1413 - val_accuracy: 0.9577
Epoch 4/20
919/919 [==============================] - 28s 30ms/step - loss: 0.1137 - accuracy: 0.9660 - val_loss: 0.1124 - val_accuracy: 0.9660
Epoch 5/20
919/919 [==============================] - 28s 30ms/step - loss: 0.0956 - accuracy: 0.9719 - val_loss: 0.1026 - val_accuracy: 0.9686
Epoch 6/20
919/919 [==============================] - 28s 30ms/step - loss: 0.0846 - accuracy: 0.9749 - val_loss: 0.0985 - val_accuracy: 0.9689
Epoch 7/20
919/919 [==============================] - 28s 30ms/step - loss: 0.0743 - accuracy: 0.9774 - val_loss: 0.0950 - val_accuracy: